#### Streaming
1. backs chunck of response

2. helps making applications based on LLMs feel responsive to end-users.

   

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

SECRET_KEY = os.environ.get("OPEN_AI_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(openai_api_key  = SECRET_KEY,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()])

In [ ]:
# Basic Streaming

chat_prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template("Tell me a fact about {topic}")
    ]
)

formatted_prompt = chat_prompt.format_messages(topic = "Python")

response  = chat.invoke(formatted_prompt)

print(response.content) 

In [ ]:
# Method 2 of streaming
chat - ChatOpenAI(openai_api_key = SECRET_KEY)
# your prompts etc..
response = chat.stream(formatted_prompt) # this returns a object of BaseChatModel.stream
for r in response:
    print(r.content, end="", flush= True)

In [ ]:
# Streaming with Chain

from langchain_core.output_parsers import StrOutputParser

chat = ChatOpenAI(openai_api_key = SECRET_KEY)
chat_prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template("Tell me a fact about {topic}")
    ]
)

chain = chat_prompt | chat | StrOutputParser()

response = chain.stream(
    {"topic" : "Python"}
)

for r in response:
    print(r, end="", flush= True)




In [ ]:
# Streaming with agents

from langchain import hub
#%pip install langchainhub
from langchain_community.tools import DuckDuckGoSearchRun
#%pip isntall -U duckduckgo-search
from langchain.agents import create_openai_functions_agent, AgentExecutor

# model
chat = ChatOpenAI(openai_api_key = SECRET_KEY)

# create Agent
prompt = hub.pull("hwchase17/openai-funtions-agent")

search = DuckDuckGoSearchRun()

tools = [search]

agent = create_openai_functions_agent(lmm = chat,
                                      tools = tools,
                                      prompt=prompt)

# Run agent
agent_executor = AgentExecutor(agent - agent,
                               tools = tools)

response = agent_executor.stream({
    "input" : "Who is the current Prime Minister of India ? "
})


print(response)
for res in response : 
    if  "output" in res : 
        print(res['output'])

# if want to know action of agents
for res in response : 
    # Agent action
    if "actions" in res:
        for action in res["actions"]:
            print(
                f"Calling Tool : '{action.tool}' with input '{action.tool_input}' "
            )
    elif "steps" in res:
        for step in res["steps"]: 
            for step in res["steps"]:
                print(f"Tool Result : `{step.observation}`")
    elif "output" in res:
        print(f"Final Output : {res['output']}")
    else :
        raise ValueError()
    print("----")
            
